In [1]:
import numpy as np
import pandas as pd
import os

from folium.features import DivIcon
import folium
import json
import dload
from shapely.geometry import shape
import geopandas as gpd

import plotly.express as px
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

# 관련 라이브러리 임포트 
import matplotlib.font_manager as fm

#  한글글꼴로 변경
# plt.rcParams['font.family'] = '한글글꼴명'
plt.rcParams['font.size'] = 11.0
# plt.rcParams['font.family'] = 'batang'
plt.rcParams['font.family'] = 'Malgun Gothic'

# 그래프에서 마이너스 폰트 깨지는 문제에 대한 대처
matplotlib.rcParams['axes.unicode_minus'] = False

# 글꼴 선명화 
%config InlineBackend.figure_format = 'retina'

# 그래프 기본 크기 설정 
# plt.rcParams['figure.figsize'] = [10, 6]

In [2]:
# 데이터
df = pd.read_csv('./data/valuable_dataset/people.csv', encoding='cp949')
df.columns = ['행정구역', '총인구', '남자', '여자']
df

,행정구역,총인구,남자,여자
0,서울특별시,9472127,4584251,4887876
1,종로구,148857,71367,77490
2,중구,126310,60657,65653
3,용산구,221688,106910,114778
4,성동구,286469,139181,147288
5,광진구,347099,166469,180630
6,동대문구,348201,169409,178792
7,중랑구,380307,186364,193943
8,성북구,435509,207428,228081
9,강북구,292611,141265,151346


In [3]:
# json 데이타 => 변수에 저장 
json_path = './data/skorea_municipalities_geo_simple.json'
json_str = open(json_path, 'r', encoding = 'utf-8')
geo_str = json.load(json_str)
type(json_str), type(geo_str)

(_io.TextIOWrapper, dict)

In [4]:
# 구별 경계점들을 지오메트리로 DF로 가져오기
center = gpd.GeoDataFrame.from_features(geo_str["features"])
center

,geometry,code,name,name_eng,base_year
0,"POLYGON ((127.11520 37.55753, 127.16683 37.576...",11250,강동구,Gangdong-gu,2013
1,"POLYGON ((127.06907 37.52228, 127.10088 37.524...",11240,송파구,Songpa-gu,2013
2,"POLYGON ((127.05867 37.52630, 127.06907 37.522...",11230,강남구,Gangnam-gu,2013
3,"POLYGON ((127.01397 37.52504, 127.03622 37.481...",11220,서초구,Seocho-gu,2013
4,"POLYGON ((126.98368 37.47386, 126.99072 37.455...",11210,관악구,Gwanak-gu,2013
5,"POLYGON ((126.98224 37.50931, 126.98368 37.473...",11200,동작구,Dongjak-gu,2013
6,"POLYGON ((126.89185 37.54737, 126.94567 37.526...",11190,영등포구,Yeongdeungpo-gu,2013
7,"POLYGON ((126.90156 37.47754, 126.91677 37.454...",11180,금천구,Geumcheon-gu,2013
8,"POLYGON ((126.82688 37.50549, 126.88156 37.513...",11170,구로구,Guro-gu,2013
9,"POLYGON ((126.79576 37.57881, 126.80702 37.601...",11160,강서구,Gangseo-gu,2013


In [5]:
# 지도형식 변환
center_new = gpd.GeoDataFrame(center, crs="EPSG:5788", geometry='geometry')
center_new

,geometry,code,name,name_eng,base_year
0,"POLYGON ((127.11520 37.55753, 127.16683 37.576...",11250,강동구,Gangdong-gu,2013
1,"POLYGON ((127.06907 37.52228, 127.10088 37.524...",11240,송파구,Songpa-gu,2013
2,"POLYGON ((127.05867 37.52630, 127.06907 37.522...",11230,강남구,Gangnam-gu,2013
3,"POLYGON ((127.01397 37.52504, 127.03622 37.481...",11220,서초구,Seocho-gu,2013
4,"POLYGON ((126.98368 37.47386, 126.99072 37.455...",11210,관악구,Gwanak-gu,2013
5,"POLYGON ((126.98224 37.50931, 126.98368 37.473...",11200,동작구,Dongjak-gu,2013
6,"POLYGON ((126.89185 37.54737, 126.94567 37.526...",11190,영등포구,Yeongdeungpo-gu,2013
7,"POLYGON ((126.90156 37.47754, 126.91677 37.454...",11180,금천구,Geumcheon-gu,2013
8,"POLYGON ((126.82688 37.50549, 126.88156 37.513...",11170,구로구,Guro-gu,2013
9,"POLYGON ((126.79576 37.57881, 126.80702 37.601...",11160,강서구,Gangseo-gu,2013


In [6]:
# 구별 중앙점을 위도 경도로 분리해서 개별 컬럼저장
center_new["long"] = center_new.to_crs(epsg='5788').centroid.map(lambda p: p.x)
center_new["lat"] = center_new.to_crs(epsg='5788').centroid.map(lambda p: p.y)

In [7]:
# 확인
center_new.head()

,geometry,code,name,name_eng,base_year,long,lat
0,"POLYGON ((127.11520 37.55753, 127.16683 37.576...",11250,강동구,Gangdong-gu,2013,127.145086,37.547123
1,"POLYGON ((127.06907 37.52228, 127.10088 37.524...",11240,송파구,Songpa-gu,2013,127.117671,37.501787
2,"POLYGON ((127.05867 37.52630, 127.06907 37.522...",11230,강남구,Gangnam-gu,2013,127.063636,37.494165
3,"POLYGON ((127.01397 37.52504, 127.03622 37.481...",11220,서초구,Seocho-gu,2013,127.031909,37.471280
4,"POLYGON ((126.98368 37.47386, 126.99072 37.455...",11210,관악구,Gwanak-gu,2013,126.946860,37.464221


In [8]:
# 좌표보정... ㅡ,.ㅡ
center_new.lat = center_new.lat - 0.011
center_new.long = center_new.long + 0.004

In [9]:
# 3) 폴리움 객체 생성 - 지도 중심 위도와 경도는 서울 
m = folium.Map(location=[37.5502, 126.982], tiles='',
                  zoom_start=11)

# # 코로폴리스 함수 호출 
m.choropleth( geo_data = geo_str,
                data = df.iloc[1:,],
                columns = ['행정구역', '총인구'],
                 key_on='feature.id',
                 fill_color='Blues' , # Purples
                 line_color='black',
                 line_weight=1.5,
                 line_opacity=0.3,
                 legend_name='구별 총인구', 
            )

for i in range(0, len(center_new)):
    folium.Marker(
      location = [center_new.iloc[i]['lat'], center_new.iloc[i]['long']],
      popup = folium.Popup(center_new.iloc[i]['name'], max_width=50),
      icon = folium.DivIcon(html=f"""<div style="font-family: courier new; color: black; font-size:100%">{center_new.iloc[i]['name']}</div>""", icon_size= (60,60)), 
   ).add_to(m)

m

C:\Users\245\AppData\Local\Programs\Python\Python310\lib\site-packages\folium\folium.py:409: FutureWarning: The choropleth  method has been deprecated. Instead use the new Choropleth class, which has the same arguments. See the example notebook 'GeoJSON_and_choropleth' for how to do this.
  warnings.warn(


In [10]:
m.save("seoul_map.html")

----
# 커피점위치
----

In [12]:
df_coffee = pd.read_csv('./data/sosang_20220630/seoul_coffee_lon_rat.csv', encoding = 'cp949')

In [14]:
df_coffee.head()

,Unnamed: 0,상권업종대분류명,상권업종중분류명,상권업종소분류명,상호명,표준산업분류명,행정동명,법정동명,경도,위도
0,20,음식,커피점/카페,커피전문점/카페/다방,커피빈,비알콜 음료점업,혜화동,명륜4가,127.000171,37.583149
1,96,음식,커피점/카페,커피전문점/카페/다방,요거프레소,비알콜 음료점업,쌍문2동,쌍문동,127.034746,37.658728
2,188,음식,커피점/카페,커피전문점/카페/다방,우성커피숍,비알콜 음료점업,신월4동,신월동,126.842935,37.522055
3,195,음식,커피점/카페,커피전문점/카페/다방,버블베어,비알콜 음료점업,방화3동,방화동,126.813358,37.580941
4,230,음식,커피점/카페,커피전문점/카페/다방,알뤼르,비알콜 음료점업,대치4동,대치동,127.054731,37.504378


In [15]:
# 3) 폴리움 객체 생성 - 지도 중심 위도와 경도는 서울 
m = folium.Map(location=[37.5502, 126.982], tiles='',
                  zoom_start=11)

# # 코로폴리스 함수 호출 
m.choropleth( geo_data = geo_str,
                data = df.iloc[1:,],
                columns = ['행정구역', '총인구'],
                 key_on='feature.id',
                 fill_color='Blues' , # Purples
                 line_color='black',
                 line_weight=1.5,
                 line_opacity=0.3,
                 legend_name='구별 총인구', 
            )

for i, j in enumerate(df_coffee):
    folium.Marker(
      location = [df_coffee[i].위도[i], df_coffee.경도[j]],
      popup = folium.Popup(df_coffee.iloc[i]['상호명'], max_width=50), 
   ).add_to(m)

m

C:\Users\245\AppData\Local\Programs\Python\Python310\lib\site-packages\folium\folium.py:409: FutureWarning: The choropleth  method has been deprecated. Instead use the new Choropleth class, which has the same arguments. See the example notebook 'GeoJSON_and_choropleth' for how to do this.
  warnings.warn(


KeyError: 'Unnamed: 0'